In [1]:
import numpy as np
import helper_functions as hf
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.metrics import categorical_accuracy, mae
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, LeakyReLU, \
    Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D, TimeDistributed, LSTM


Using TensorFlow backend.


In [2]:
X_train_eeg1, _ = hf.read_csv_to_matrix("task5/input/train_eeg1.csv", "Id")
X_train_eeg2, _ = hf.read_csv_to_matrix("task5/input/train_eeg2.csv", "Id")
X_train_emg, _ = hf.read_csv_to_matrix("task5/input/train_emg.csv", "Id")
X_test_eeg1, _ = hf.read_csv_to_matrix("task5/input/test_eeg1.csv", "Id")
X_test_eeg2, _ = hf.read_csv_to_matrix("task5/input/test_eeg2.csv", "Id")
X_test_emg, _ = hf.read_csv_to_matrix("task5/input/test_emg.csv", "Id")
y_train, _ = hf.read_csv_to_matrix("task5/input/train_labels.csv", "Id")
_, test_index = hf.read_csv_to_matrix("task5/input/sample.csv", "Id")
y_train = np.squeeze(y_train)


In [3]:
def cnn_model():
    filt = 3
    
    model = Sequential()

    model.add(Conv1D(name="conv1", filters=2**filt, kernel_size=3, input_shape=(1536, 1)))
    model.add(BatchNormalization(name="cnorm1"))
    model.add(LeakyReLU(name="act1", alpha=.01))
    model.add(MaxPooling1D(name="mp1", pool_size=2))
    
    model.add(Conv1D(name="conv2", filters=2**(filt+1), kernel_size=3))
    model.add(BatchNormalization(name="cnorm2"))
    model.add(LeakyReLU(name="act2", alpha=.01))
    model.add(MaxPooling1D(name="mp2", pool_size=2))

    model.add(Conv1D(name="conv3", filters=2**(filt+2), kernel_size=3))
    model.add(BatchNormalization(name="cnorm3"))
    model.add(LeakyReLU(name="act3", alpha=.01))
    model.add(MaxPooling1D(name="mp3", pool_size=2))

    model.add(Conv1D(name="conv4", filters=2**(filt+3), kernel_size=3))
    model.add(BatchNormalization(name="cnorm4"))
    model.add(LeakyReLU(name="act4", alpha=.01))
    model.add(MaxPooling1D(name="mp4", pool_size=2))
    
    model.add(Conv1D(name="conv5", filters=2**(filt+4), kernel_size=3))
    model.add(BatchNormalization(name="cnorm5"))
    model.add(LeakyReLU(name="act5", alpha=.01))
    model.add(MaxPooling1D(name="mp5", pool_size=2))
    
    model.add(Conv1D(name="conv6", filters=2**(filt+5), kernel_size=3))
    model.add(BatchNormalization(name="cnorm6"))
    model.add(LeakyReLU(name="act6", alpha=.01))
    model.add(MaxPooling1D(name="mp6", pool_size=2))
    
    model.add(Conv1D(name="conv7", filters=2**(filt+6), kernel_size=3))
    model.add(BatchNormalization(name="cnorm7"))
    model.add(LeakyReLU(name="act7", alpha=.01))
    model.add(MaxPooling1D(name="mp7", pool_size=2))
    
    model.add(Conv1D(name="conv8", filters=2**(filt+7), kernel_size=3))
    model.add(BatchNormalization(name="cnorm8"))
    model.add(LeakyReLU(name="act8", alpha=.01))
    
    model.add(GlobalAveragePooling1D(name="mp8"))
    
    # model.add(Flatten(name="flat1"))
    
    model.add(Dense(name="full1", units=1024))
    model.add(BatchNormalization(name="fnorm1"))
    model.add(LeakyReLU(name="dact1", alpha=.01))
    model.add(Dropout(name="fdrop1", rate=0.5))
    
    model.add(Dense(name="full2", units=512))
    model.add(BatchNormalization(name="fnorm2"))
    model.add(LeakyReLU(name="dact2", alpha=.01))
    model.add(Dropout(name="fdrop2", rate=0.25))
    
    model.add(Dense(name="full3", units=2048))
    model.add(BatchNormalization(name="fnorm3"))
    model.add(LeakyReLU(name="dact3", alpha=.01))
    model.add(Dropout(name="fdrop3", rate=0.5))
    
    model.add(Dense(name="full4", units=256))
    model.add(BatchNormalization(name="fnorm4"))
    model.add(LeakyReLU(name="dact4", alpha=.01))
    model.add(Dropout(name="fdrop4", rate=0.25))
    
    model.add(Dense(name="full5", units=64))
    model.add(BatchNormalization(name="fnorm5"))
    model.add(LeakyReLU(name="dact5", alpha=.01))
    model.add(Dropout(name="fdrop5", rate=0.5))
    
    model.add(Dense(name="full6", units=128))
    model.add(BatchNormalization(name="fnorm6"))
    model.add(LeakyReLU(name="dact6", alpha=.01))
    model.add(Dropout(name="fdrop6", rate=0.25))
    
    model.add(Dense(name="last", units=3, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[mae, categorical_accuracy])
    return model


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
X_train = np.c_[X_train_eeg1, X_train_eeg2, X_train_emg]

ss = StandardScaler()
X_train = ss.fit_transform(X_train)

size = 1536
"""

pca = PCA(n_components=size)
X_train = pca.fit_transform(X_train)
"""

X_train = X_train.reshape(-1, size, 1)
"""

X = X_train[0:21600].reshape(-1, 21600, size, 1)
X = np.append(X, X_train[21600:43200].reshape(-1, 21600, size, 1), axis=0)
X = np.append(X, X_train[43200:64800].reshape(-1, 21600, size, 1), axis=0)
X_train = X
"""


'\n\nX = X_train[0:21600].reshape(-1, 21600, size, 1)\nX = np.append(X, X_train[21600:43200].reshape(-1, 21600, size, 1), axis=0)\nX = np.append(X, X_train[43200:64800].reshape(-1, 21600, size, 1), axis=0)\nX_train = X\n'

In [5]:
clf = KerasClassifier(build_fn=cnn_model, epochs=2, batch_size=1000, verbose=1)

print("========= CrossValidation =========")
results = cross_val_score(estimator=clf, X=X_train, y=y_train, cv=3, n_jobs=1, scoring=hf.scorer(), 
                          pre_dispatch='n_jobs', verbose=1)
print("Results: %.4f (%.4f) MSE" % (results.mean(), results.std()))


========= CrossValidation =========


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/2


 1000/43200 [..............................] - ETA: 7:16 - loss: 1.4310 - mean_absolute_error: 0.4596 - categorical_accuracy: 0.2930

 2000/43200 [>.............................] - ETA: 5:58 - loss: 1.2891 - mean_absolute_error: 0.4357 - categorical_accuracy: 0.3655

...

20000/21600 [==========================>...] - ETA: 2s

21000/21600 [============================>.] - ETA: 0s

21600/21600 [==============================] - 35s 2ms/step


Results: 0.8735 (0.0509) MSE


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 33.2min finished


In [11]:
from sklearn.utils import compute_sample_weight
X_test = np.c_[X_test_eeg1, X_test_eeg2, X_test_emg]
X_test = ss.transform(X_test)
#X_test = pca.transform(X_test)
X_test = X_test.reshape(-1, size, 1)

clf = KerasClassifier(build_fn=cnn_model, epochs=3, batch_size=4000, verbose=1)
clf.fit(X_train, y_train)
# clf.fit(X_train, y_train, sample_weight=compute_sample_weight(class_weight='balanced', y=y_train))
y_pred = clf.predict(X_test)

print("==> Printing solution")
hf.write_to_csv_from_vector("solution_franz_1.csv", test_index, y_pred, "Id")

Epoch 1/3


 4000/64800 [>.............................] - ETA: 12:23 - loss: 1.4452 - mean_absolute_error: 0.4588 - categorical_accuracy: 0.2935

 8000/64800 [==>...........................] - ETA: 10:05 - loss: 1.3095 - mean_absolute_error: 0.4388 - categorical_accuracy: 0.3474

12000/64800 [====>.........................] - ETA: 8:43 - loss: 1.1982 - mean_absolute_error: 0.4189 - categorical_accuracy: 0.4003 

16000/64800 [======>.......................] - ETA: 7:49 - loss: 1.0963 - mean_absolute_error: 0.3961 - categorical_accuracy: 0.4589

...

32000/43200 [=====================>........] - ETA: 36s

36000/43200 [========================>.....] - ETA: 22s

40000/43200 [==========================>...] - ETA: 9s 

43200/43200 [==============================] - 128s 3ms/step


==> Printing solution
